##### Thinking1	电商定向广告和搜索广告有怎样的区别，算法模型是否有差别
答：定向广告为个性化推荐,即我们常提到的千人千面或者千人十面，就是在用户使用系统的时候并没有进行特定商品的搜索，这时候电商系统去根据用户可能的需要进行商品推荐，就叫做定向广告。当用户进入系统带着明确的购买目标，在浏览中产生明确的搜索和点击行为时，这时候的广告推送为搜索广告的模式。定向广告算法模型会去根据用户的历史行为信息及用户统计特征去挖掘用户可能会产生兴趣甚至转化为购买行为的商品，商品类别可能会相对多一点，二搜索广告由于用户具有明确的购买目标，只需要根据用户当前的目标去推荐同类型产品即可。
##### Thinking2	定向广告都有哪些常见的使用模型，包括Attention机制模型
答：LR模型,MLR（混合线性模型）,DNN（深度模型），DIN(Deep Interest Network),DIEN(Deep Interest Evolution Network),DSIN(Deep Session Interest Network)。
##### Thinking3	DIN中的Attention机制思想和原理是怎样的
答：在对用户查看过的所有历史商品做SUM Pooling的时候不再采用DNN中简单的特征对位相加，而是考虑到用户当前的注意力，用历史行为去和候选广告进行匹配计算，得到每个商品的权重，在计算SUM Pooling的时候引入这个权重即Activation Unit，对于不同的候选广告，得到的用户行为表示向量是不同的，这样就能更加关注当前用户的兴趣点。
##### Thinking4	DIEN相比于DIN有哪些创新
答：DIEN中增加了AUGRU循环神经网络机制，利用GRU中时间产生的序列行为建模，捕捉行为之间的依赖，模拟用户兴趣的演化过程，从而学习用户的兴趣迁移。
##### Thinking5	DSIN关于Session的洞察是怎样的，如何对Session兴趣进行表达
答：用户在每个Session内的行为是相近的，在时序相差较远的两个Session之间的行为特征是相差较大的，DSIN用自注意力机制提取用户的Session兴趣向量，用Bi-LSTM对用户的Session之间的交互提取带有上下文信息的Session兴趣向量，然后利用局部激活单元自适应地学习各种Session兴趣对目标item的影响。
##### Thinking6	如果你来设计淘宝定向广告，会有哪些future work（即下一个阶段的idea）
答：在淘宝中，有些品类的商品是属于生活消耗品，例如卫生用品等，是一个不断消耗的品类，每个用户会根据自己的家庭需求周期性的采购，可以把这种特征加上，周期性地向用户推荐这种类别的产品。


In [69]:
import pandas as pd
# 加载小样本
'''
user_log = pd.read_csv('./sample_user_log.csv', dtype={'time_stamp':'str'})
user_info = pd.read_csv('./sample_user_info.csv')
train_data1 = pd.read_csv('./train.csv')
submission = pd.read_csv('./test.csv')
'''
user_log = pd.read_csv('./user_log_format1.csv', dtype={'time_stamp':'str'})
user_info = pd.read_csv('./user_info_format1.csv')
train_data1 = pd.read_csv('./train_format1.csv')
submission = pd.read_csv('./test_format1.csv')

In [70]:
print(user_log.head(5))
print(user_info.head(5))
print(train_data1.head(5))
print(submission.head(5))
print("user_log-----------")
print(user_log.info())
print("user_info-----------")
print(user_info.info())
print("train_data1-----------")
print(train_data1.info())
print("submission-----------")
print(submission.info())

   user_id  item_id  cat_id  seller_id  brand_id time_stamp  action_type
0   328862   323294     833       2882    2661.0       0829            0
1   328862   844400    1271       2882    2661.0       0829            0
2   328862   575153    1271       2882    2661.0       0829            0
3   328862   996875    1271       2882    2661.0       0829            0
4   328862  1086186    1271       1253    1049.0       0829            0
   user_id  age_range  gender
0   376517        6.0     1.0
1   234512        5.0     0.0
2   344532        5.0     0.0
3   186135        5.0     0.0
4    30230        5.0     0.0
   user_id  merchant_id  label
0    34176         3906      0
1    34176          121      0
2    34176         4356      1
3    34176         2217      0
4   230784         4818      0
   user_id  merchant_id  prob
0   163968         4605   NaN
1   360576         1581   NaN
2    98688         1964   NaN
3    98688         3645   NaN
4   295296         3361   NaN
user_log--------

In [71]:
train_data = pd.read_csv('./train_format2.csv')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7030723 entries, 0 to 7030722
Data columns (total 6 columns):
 #   Column        Dtype  
---  ------        -----  
 0   user_id       int64  
 1   age_range     float64
 2   gender        float64
 3   merchant_id   int64  
 4   label         int64  
 5   activity_log  object 
dtypes: float64(2), int64(3), object(1)
memory usage: 321.8+ MB


In [72]:
#train_data1,submission重增加origin字段，用来区别是traindata还是testdata,然后将train和test拼接成一张大表
train_data1['origin'] = 'train'
submission['origin'] = 'test'
matrix = pd.concat([train_data1, submission], ignore_index=True, sort=False)

In [73]:
matrix.drop(['prob'], axis=1, inplace=True)
# 连接user_info表，通过user_id关联
matrix = matrix.merge(user_info, on='user_id', how='left')

In [74]:
# 使用merchant_id（原列名seller_id）,将不同表之间相同属性列名进行统一
user_log.rename(columns={'seller_id':'merchant_id'}, inplace=True)
# 格式化，修改id属性的字段类型
user_log['user_id'] = user_log['user_id'].astype('int32')
user_log['merchant_id'] = user_log['merchant_id'].astype('int32')
user_log['item_id'] = user_log['item_id'].astype('int32')
user_log['cat_id'] = user_log['cat_id'].astype('int32')
#填充空值
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype('int32')
#对时间数据进行格式化
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')

In [75]:
# 1 for <18; 2 for [18,24]; 3 for [25,29]; 4 for [30,34]; 5 for [35,39]; 6 for [40,49]; 7 and 8 for >= 50; 0 and NULL for unknown
#0填充年龄空值
matrix['age_range'].fillna(0, inplace=True)
# 0:female, 1:male, 2:unknown
#2填充性别空值
matrix['gender'].fillna(2, inplace=True)
#修改字段类型
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')
matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')

In [76]:
import gc
#删除变量user_info，train_data1
del user_info, train_data1
gc.collect()

148

In [77]:
# User特征处理
groups = user_log.groupby(['user_id'])

In [78]:
# 统计用户交互行为数量 并将该字段命名为u1
temp = groups.size().reset_index().rename(columns={0:'u1'})

In [79]:
#将u1字段合并到主表中
matrix = matrix.merge(temp, on='user_id', how='left')

In [80]:
#统计用户行为，包括用户购买的商品，购买的种类，购买的品牌，购买过的商店
#统计每个user_id中有多少个唯一item_id数，并将该字段设置为u2
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
# 对于每个user_id 不重复的cat_id的数量 => u3
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')

In [81]:
#计算用户发生购买行为的时间间隔
# 时间间隔特征 u6 按照小时
# 对于每个user_id 计算time_stamp的最小时间 => F_time, 最大时间max => L_time
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
# 统计操作类型为0，1，2，3的个数，并设置列名
#unstack: 数据从”花括号结构“变成”表格结构
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')

In [82]:
matrix

,user_id,merchant_id,label,origin,age_range,gender,u1,u2,u3,u4,u5,u6,u7,u8,u9,u10
0,34176,3906,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0
1,34176,121,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0
2,34176,4356,1.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0
3,34176,2217,0.0,train,6,0,451,256,45,109,108,5.833333,410.0,NaN,34.0,7.0
4,230784,4818,0.0,train,0,0,54,31,17,20,19,5.166667,47.0,NaN,7.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522336,228479,3111,nan,test,6,0,2004,1173,71,278,282,6.000000,1770.0,NaN,26.0,208.0
522337,97919,2341,nan,test,8,1,55,29,14,17,17,4.750000,46.0,NaN,8.0,1.0
522338,97919,3971,nan,test,8,1,55,29,14,17,17,4.750000,46.0,NaN,8.0,1.0
522339,32639,3536,nan,test,0,0,72,46,24,33,35,5.800000,62.0,1.0,8.0,1.0


In [83]:
# 商家特征处理，包括商家拥有的交互次数，商家被交互的user_id, item_id, cat_id, brand_id、action_type 唯一值
groups = user_log.groupby(['merchant_id'])

temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id':'m2', 'item_id':'m3', 'cat_id':'m4', 'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 按照merchant_id 统计随机负采样的个数
temp = train_data[train_data['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [84]:
matrix.columns

Index(['user_id', 'merchant_id', 'label', 'origin', 'age_range', 'gender',
       'u1', 'u2', 'u3', 'u4', 'u5', 'u6', 'u7', 'u8', 'u9', 'u10', 'm1', 'm2',
       'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10'],
      dtype='object')

In [85]:
# 按照user_id, merchant_id分组进行统计各项指标的数量
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'}) #统计行为个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'um2', 'cat_id':'um3', 'brand_id':'um4'}) #统计item_id, cat_id, brand_id唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'um5', 1:'um6', 2:'um7', 3:'um8'})#统计不同action_type唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['first', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left') #统计时间间隔

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


In [86]:
#用户购买点击比，u9为该用户的购买行为次数，u7为用户单击次数
matrix['r1'] = matrix['u9']/matrix['u7'] 
#商家购买点击比，m8为该商家商品被购买的次数，m6为该商家被点击的次数
matrix['r2'] = matrix['m8']/matrix['m6'] 
#不同用户不同商家购买点击比，um7为同一用户对同一商家产生的购买行为次数，um5为同一用户对同一商家产生的点击行为次数
matrix['r3'] = matrix['um7']/matrix['um5']
matrix.fillna(0, inplace=True)
#对age_range、gender进行编码
# # 修改age_range字段名称为 age_0, age_1, age_2... age_8
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)
print(matrix)

        user_id  merchant_id label origin    u1    u2  u3   u4   u5        u6  \
0         34176         3906   0.0  train   451   256  45  109  108  5.833333   
1         34176          121   0.0  train   451   256  45  109  108  5.833333   
2         34176         4356   1.0  train   451   256  45  109  108  5.833333   
3         34176         2217   0.0  train   451   256  45  109  108  5.833333   
4        230784         4818   0.0  train    54    31  17   20   19  5.166667   
...         ...          ...   ...    ...   ...   ...  ..  ...  ...       ...   
522336   228479         3111   nan   test  2004  1173  71  278  282  6.000000   
522337    97919         2341   nan   test    55    29  14   17   17  4.750000   
522338    97919         3971   nan   test    55    29  14   17   17  4.750000   
522339    32639         3536   nan   test    72    46  24   33   35  5.800000   
522340    32639         3319   nan   test    72    46  24   33   35  5.800000   

        ...  age_2  age_3  

In [87]:

# 根据之前设置的origin标记字段分割训练数据和测试数据
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']
del temp, matrix
gc.collect()

51

In [88]:
# 使用XGBoost
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
import xgboost as xgb

X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.2)
model = xgb.XGBClassifier(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42    
)
model.fit(
    X_train, y_train,
    eval_metric='auc', eval_set=[(X_train, y_train), (X_valid, y_valid)],
    verbose=True,
    #早停法，如果auc在10epoch没有进步就stop
    early_stopping_rounds=10 
)


model.fit(X_train, y_train)

prob = model.predict_proba(test_data)
submission['prob'] = pd.Series(prob[:,1])
submission.drop(['origin'], axis=1, inplace=True)
submission.to_csv('prediction.csv', index=False)

[0]	validation_0-auc:0.63759	validation_1-auc:0.62797
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.65080	validation_1-auc:0.64002
[2]	validation_0-auc:0.65670	validation_1-auc:0.64299
[3]	validation_0-auc:0.66342	validation_1-auc:0.64542
[4]	validation_0-auc:0.66469	validation_1-auc:0.64665
[5]	validation_0-auc:0.66569	validation_1-auc:0.64732
[6]	validation_0-auc:0.66820	validation_1-auc:0.64840
[7]	validation_0-auc:0.66856	validation_1-auc:0.64924
[8]	validation_0-auc:0.67073	validation_1-auc:0.65161
[9]	validation_0-auc:0.67120	validation_1-auc:0.65333
[10]	validation_0-auc:0.67274	validation_1-auc:0.65500
[11]	validation_0-auc:0.67362	validation_1-auc:0.65578
[12]	validation_0-auc:0.67533	validation_1-auc:0.65634
[13]	validation_0-auc:0.67676	validation_1-auc:0.65809
[14]	validation_0-auc:0.67934	validation_1-auc:0.65851
[15]	validation_0-auc:0.6811